####Connect with google drive


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Take in dataset from: [Popular Youtube Channels Dataset](https://www.kaggle.com/datasets/zhitaochen/channels)

In [ ]:
import pandas as pd
import csv
data = pd.read_csv("./gdrive/MyDrive/youtube_thumbnails/channels.csv", encoding='utf-8',on_bad_lines='skip',engine="python")

In [ ]:
data

,id,url,description
0,UCMdRkWIe9mdpsw4-8EID4rw,https://www.youtube.com/channel/UCMdRkWIe9mdps...,Movies. Music. Entertainment. Justified - Open...
1,UCUsTSbCLD96-pcWi2Kqaitg,https://www.youtube.com/channel/UCUsTSbCLD96-p...,Lifestyle. Aussie Comedian. Gaming Video Creat...
2,UCyCyTe_1bT2aIPwG_gxbyeg,https://www.youtube.com/channel/UCyCyTe_1bT2aI...,Lifestyle. The Lanning family has been making ...
3,UCHuD3bhRhvOuAPqVJZlQfWw,https://www.youtube.com/channel/UCHuD3bhRhvOuA...,Entertainment. Movies. N.R.P Cinemas is a mark...
4,UCW2OUlFrrWiZvSsZRwOYmNg,https://www.youtube.com/channel/UCW2OUlFrrWiZv...,Vehicles. Lifestyle. YOUCAR is the world's fir...
...,...,...,...
16378,UC4SUWizzKc1tptprBkWjX2Q,https://www.youtube.com/channel/UC4SUWizzKc1tp...,Society. Coco Lee remembered as hundreds of mo...
16379,UCVNFzYxySA5UFvAi32vD0cg,https://www.youtube.com/channel/UCVNFzYxySA5UF...,Vehicles. Lifestyle. reXpair brings evolution ...
16380,UC3hhq3Twc_jTYWzfL5BCa1A,https://www.youtube.com/channel/UC3hhq3Twc_jTY...,Music. Pop music. Official YouTube channel for...
16381,UCAt0ZufG7-TvfgbwkCbKd3Q,https://www.youtube.com/channel/UCAt0ZufG7-Tvf...,Entertainment. Movies. Hello Guys I m Paritosh...


In [ ]:
len(data)

16383

####Download titles and thumbnails_links into all_urls.pickle

In [ ]:
%pip install pytubefix
%pip install progressbar2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
from pytubefix import Channel,YouTube
from pytubefix.cli import on_progress
from rich.progress import Progress, BarColumn, TaskProgressColumn, TimeRemainingColumn,MofNCompleteColumn,TimeElapsedColumn,TextColumn
from time import sleep
import pickle

In [ ]:
url_channel=data["url"][0]
channel=Channel(url_channel)
print(url_channel)

https://www.youtube.com/channel/UCMdRkWIe9mdpsw4-8EID4rw


In [ ]:
def load_data_to_file(data):
  with open('./gdrive/MyDrive/youtube_thumbnails/all_urls.pickle', 'a+b') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
  handle.close()
def create_chunk_from_channel_url(channel_url,progress_bar):
  try:
    channel=Channel(channel_url)
    chunk_data={"description":channel.description.partition('\n'),
          "channel_name":channel.channel_name,
          "channel_url":channel_url,
          "thumb_title":[]}
    t1 = progress_bar.add_task( total=int(channel.length.split(" ")[0]),description=channel_url, start=True, )
    for j,video in enumerate(channel.videos):
      progress_bar.update(task_id=t1, completed=j+1)
      try:
          yt = YouTube(video.watch_url, use_oauth=True, allow_oauth_cache=True, on_progress_callback = on_progress)
      except:
            continue
      chunk_data["thumb_title"].append((yt.title,yt.thumbnail_url,yt.views))
    progress_bar.update(task_id=t1, completed=int(channel.length.split(" ")[0]))
    progress_bar.remove_task(t1,)

    return chunk_data
  except:
    return None


In [ ]:
# with open('./gdrive/MyDrive/youtube_thumbnails/configuration.pickle', 'wb') as handle:#обновляет значение
#     pickle.dump({"channel_id":0}, handle, protocol=pickle.HIGHEST_PROTOCOL)
# open('all_urls.pickle', 'wb').close()

In [ ]:
with open('./gdrive/MyDrive/youtube_thumbnails/configuration.pickle', 'rb') as handle:
    configuration = pickle.load(handle)
    handle.close()
from rich.progress import Progress

with Progress(MofNCompleteColumn(),
              BarColumn(),
              TaskProgressColumn(),
              TimeRemainingColumn(elapsed_when_finished=True),
              TimeElapsedColumn(),
              TextColumn(" [bold blue]{task.description}", justify="right"),
              transient=True) as pb:
    indexes_loaded=configuration["channel_id"]
    t2 = pb.add_task(total=len(data["url"])-indexes_loaded,description="",start=True)
    for i,url_channel in enumerate(data["url"]):
      if(i<configuration["channel_id"]):
        continue
      chunk_data=create_chunk_from_channel_url(url_channel,pb)
      if chunk_data!=None:
        load_data_to_file(chunk_data)
      with open('./gdrive/MyDrive/youtube_thumbnails/configuration.pickle', 'wb') as handle:
        pickle.dump(configuration, handle, protocol=pickle.HIGHEST_PROTOCOL)
        handle.close()
      configuration["channel_id"]=i+1
      sleep(0.01)
      pb.update(task_id=t2, completed=i + 1-indexes_loaded)

Output()

Please open https://www.google.com/device and input code RJZ-CQX-SWHS

Press enter when you have completed this step.


In [ ]:
import pickle
data_thumbnail = []
with open('./gdrive/MyDrive/youtube_thumbnails/all_urls.pickle', 'rb') as fr:
    try:
        while True:
            data_thumbnail.append(pickle.load(fr))
    except EOFError:
        pass

In [ ]:
data_thumbnail[0]

{'description': ("My name is JR Alli. I'm a 26 year old Filmmaker from Toronto, Canada. Click that Subscribe button if you enjoy short films or adventures.",
  '\n',
  '\nI want to share with you my thoughts on how we as humans need to start living life in the moment. I just want to inspire other people to go out into the world and see what it has to offer, and I hope to achieve this through my Filmmaking. If you are interested, come along for the ride and subscribe! :)\n\n__________________________________________________________________________________________________'),
 'channel_name': 'JR Alli',
 'channel_url': 'https://www.youtube.com/channel/UC94IKgdvTjCqoKqSlbj2bVA',
 'thumb_title': [('I travelled alone to Tokyo.',
   'https://i.ytimg.com/vi/cTT_VcZYJuw/sddefault.jpg',
   20705),
  ('The Age of Addiction',
   'https://i.ytimg.com/vi/oyGmpCJJfHU/sddefault.jpg',
   16284),
  ('AROUND THE WORLD 2023',
   'https://i.ytimg.com/vi/NTlEQRZzd4A/sddefault.jpg',
   17218),
  ('a MOMENT i

####Filter data to have english title

In [ ]:
counter=0
for i in data_thumbnail:
  counter+=len(i["thumb_title"])
counter

2567725

In [ ]:
%pip install langdetect

In [ ]:
open('./gdrive/MyDrive/youtube_thumbnails/training_data.csv', 'w', newline='').close()

In [ ]:
import csv
from tqdm import tqdm
from langdetect import detect
with open('./gdrive/MyDrive/youtube_thumbnails/training_data.csv', 'w', newline='') as file:
    writer = csv.writer(file,escapechar='\\')
    field = ["channel_description", "channel_name", "channel_url","title","thumb_url","views"]
    writer.writerow(field)
    for channel in tqdm(data_thumbnail):
      channel_description,channel_name,channel_url=channel["description"][0],channel["channel_name"],channel["channel_url"]
      counter=0
      for title,thumbnail_url,views in channel["thumb_title"]:
        try:
          if(detect(title)=="en"):
            writer.writerow([channel_description,channel_name,channel_url,title,thumbnail_url,views])
          else:
            counter+=1
            if(counter>=10):#you can choose any other number
              break
        except:
          print("This row throws and error:", channel_url, title)

  1%|          | 59/10879 [00:38<1:50:44,  1.63it/s]

This row throws and error: https://www.youtube.com/channel/UCiJfplrc7idtWYpDsI325yQ ට්‍රිප් එකක් ගිය වෙලාවෙ ගෙදරට හොරු පැනලා. CCTV දර්ශන සහිතයි.


  1%|          | 93/10879 [01:00<1:54:30,  1.57it/s]

This row throws and error: https://www.youtube.com/channel/UCT2ZNwfdGCRQPYCK0ZvCWsg #2020


  2%|▏         | 182/10879 [02:07<1:50:39,  1.61it/s]

...

 99%|█████████▉| 10774/10879 [2:19:59<02:34,  1.47s/it]

This row throws and error: https://www.youtube.com/channel/UCW67rDTapVxGZ69Ja9i6Ifw 3 ❤️


100%|█████████▉| 10867/10879 [2:21:09<00:03,  3.51it/s]

This row throws and error: https://www.youtube.com/channel/UCz66sJbzTTte0_lX6rzkK5A )


100%|██████████| 10879/10879 [2:21:14<00:00,  1.28it/s]


In [ ]:
import pandas as pd
Data=pd.read_csv('./gdrive/MyDrive/youtube_thumbnails/training_data.csv')
Data

,channel_description,channel_name,channel_url,title,thumb_url,views
0,My name is JR Alli. I'm a 26 year old Filmmake...,JR Alli,https://www.youtube.com/channel/UC94IKgdvTjCqo...,The Age of Addiction,https://i.ytimg.com/vi/oyGmpCJJfHU/sddefault.jpg,16284
1,My name is JR Alli. I'm a 26 year old Filmmake...,JR Alli,https://www.youtube.com/channel/UC94IKgdvTjCqo...,AROUND THE WORLD 2023,https://i.ytimg.com/vi/NTlEQRZzd4A/sddefault.jpg,17218
2,My name is JR Alli. I'm a 26 year old Filmmake...,JR Alli,https://www.youtube.com/channel/UC94IKgdvTjCqo...,the Memory Palace,https://i.ytimg.com/vi/lQfadWsNIXQ/sddefault.jpg,83667
3,My name is JR Alli. I'm a 26 year old Filmmake...,JR Alli,https://www.youtube.com/channel/UC94IKgdvTjCqo...,"Dear Iceland,",https://i.ytimg.com/vi/f-fJ6aav-Cs/sddefault.jpg,170228
4,My name is JR Alli. I'm a 26 year old Filmmake...,JR Alli,https://www.youtube.com/channel/UC94IKgdvTjCqo...,RONIN-SC Hands On Review in TALLINN!,https://i.ytimg.com/vi/KYzk7Br6vd4/sddefault.jpg,69699
...,...,...,...,...,...,...
795946,Chasing Summer Productions is a team of videog...,Chasing Summer Productions,https://www.youtube.com/channel/UCLogORgVX9dA3...,Dummy Mount GoPro's Most Versatile Mount,https://i.ytimg.com/vi/fD0ptaUxYLY/sddefault.jpg,70925
795947,Chasing Summer Productions is a team of videog...,Chasing Summer Productions,https://www.youtube.com/channel/UCLogORgVX9dA3...,Cliff Dive Hawaii Oahu,https://i.ytimg.com/vi/5OPbI2f9Qtw/sddefault.j...,3789
795948,Chasing Summer Productions is a team of videog...,Chasing Summer Productions,https://www.youtube.com/channel/UCLogORgVX9dA3...,Cliff Diving Hawaii Maui & Big Island,https://i.ytimg.com/vi/cUmCPaVhul4/sddefault.jpg,257435
795949,Chasing Summer Productions is a team of videog...,Chasing Summer Productions,https://www.youtube.com/channel/UCLogORgVX9dA3...,Maui Cliff Diving,https://i.ytimg.com/vi/3y8AHGm60-4/sddefault.j...,2152
